In [26]:
import json
import re

poem = "nymphs_reply"
poem_path = f"../static/poems/{poem}/fulltext.txt"
source_poem = open(poem_path, "r").read()
stanza_no = 3
source_mp3 = f"../static/poems/{poem}/stanza{stanza_no}.mp3"
source_wav = f"../static/poems/{poem}/stanza{stanza_no}.wav"
MODEL_PATH = "./vosk-model-small-en-us-0.15"
source_transcript = source_poem.split("\n\n")[stanza_no - 1].replace("'s", "")
grammar_words = re.findall(r'\w+', source_transcript.lower()) # + ["[unk]"]
# grammar_str = json.dumps(grammar_words)
grammar_str = json.dumps([" ".join(grammar_words)])
source_transcript
grammar_str

'["the flowers do fade and wanton fields to wayward winter reckoning yields a honey tongue a heart of gall is fancy spring but sorrow fall"]'

In [27]:
import vosk
import wave
import json

timestamps = []


wf = wave.open(source_wav, "rb")
model = vosk.Model(MODEL_PATH)
recognizer = vosk.KaldiRecognizer(model, wf.getframerate())
recognizer.SetWords(True)
recognizer.SetGrammar(grammar_str)

while True:
    data = wf.readframes(4000)
    if recognizer.AcceptWaveform(data):
        result = json.loads(recognizer.Result())
        words = result.get("result", [])
        timestamps += words
    else:
        pass
    if len(data) == 0:
        break
# finalize the recognizer
result = recognizer.FinalResult()
result = json.loads(result)
words = result.get("result", [])
timestamps += words
print(timestamps)
for i, timestamp in enumerate(timestamps):
    print(i, timestamp["word"], timestamp["start"], timestamp["end"])

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from ./vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from ./vosk-model-small-en-us-0.15/graph/HCLr.fst ./vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo ./vosk-model-small-en-us-0.15/graph/phones/word_boundary.int


[{'conf': 1.0, 'end': 0.66, 'start': 0.51, 'word': 'the'}, {'conf': 1.0, 'end': 1.02, 'start': 0.66, 'word': 'flowers'}, {'conf': 1.0, 'end': 1.14, 'start': 1.02, 'word': 'do'}, {'conf': 1.0, 'end': 1.53, 'start': 1.14, 'word': 'fade'}, {'conf': 1.0, 'end': 1.65, 'start': 1.53, 'word': 'and'}, {'conf': 1.0, 'end': 2.07, 'start': 1.65, 'word': 'wanton'}, {'conf': 1.0, 'end': 2.67, 'start': 2.07, 'word': 'fields'}, {'conf': 1.0, 'end': 2.82, 'start': 2.67, 'word': 'to'}, {'conf': 1.0, 'end': 3.27, 'start': 2.82, 'word': 'wayward'}, {'conf': 1.0, 'end': 3.66, 'start': 3.27, 'word': 'winter'}, {'conf': 1.0, 'end': 4.08, 'start': 3.66, 'word': 'reckoning'}, {'conf': 1.0, 'end': 4.71, 'start': 4.08, 'word': 'yields'}, {'conf': 1.0, 'end': 5.04, 'start': 4.92, 'word': 'a'}, {'conf': 1.0, 'end': 5.34, 'start': 5.04, 'word': 'honey'}, {'conf': 1.0, 'end': 5.82, 'start': 5.34, 'word': 'tongue'}, {'conf': 1.0, 'end': 5.97, 'start': 5.85, 'word': 'a'}, {'conf': 1.0, 'end': 6.27, 'start': 5.97, 'wo

LOG (VoskAPI:UpdateGrammarFst():recognizer.cc:287) ["the flowers do fade and wanton fields to wayward winter reckoning yields a honey tongue a heart of gall is fancy spring but sorrow fall"]
LOG (VoskAPI:Estimate():language_model.cc:142) Estimating language model with ngram-order=2, discount=0.5
LOG (VoskAPI:OutputToFst():language_model.cc:209) Created language model with 25 states and 72 arcs.


In [28]:
timestamps2 = [entry.copy() for entry in timestamps]
# timestamps2[11]["word"] = "kirtle"
# timestamps2[15]["word"] = "posies"
# timestamps2.pop(12)
# timestamps2[17]["word"] = "becometh"
# timestamps2.pop(18)
# timestamps2[5]["word"] = "trunkless"
# timestamps2[5]["end"] = timestamps2[7]["end"]
# timestamps2.pop(6)
# timestamps2.pop(6)
# timestamps2 = timestamps2[:10] + [{"word": "a", "start": 4.31, "end": 4.40, "conf": 1.0}] + timestamps2[10:]
# timestamps2.pop(21)
# timestamps2[-1]["end"] += 0.25
len(timestamps2)

25

In [29]:
outfile = f"../static/poems/{poem}/stanza{stanza_no}_timestamps.json"
with open(outfile, "w") as f:
    json.dump(timestamps2, f, indent=4)